In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam

# Check file existence
filename = 'Libro1KNVS.xlsx'
# Read data from Excel
datosDF = pd.read_excel(filename)
df = pd.DataFrame(datosDF)

In [ ]:
ss = StandardScaler()
scaler = ss.fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [ ]:
numbers_of_rows = datosDF.values.shape[0]
window_length = 14
number_of_features = datosDF.values.shape[1]
train = np.empty([numbers_of_rows-window_length, window_length, number_of_features], dtype=float)
label = np.empty([numbers_of_rows-window_length, number_of_features], dtype=float)

for i in range(0, numbers_of_rows-window_length):
    train[i] = transformed_df.iloc[i+window_length, 0:number_of_features]
    label[i] = transformed_df.iloc[i+window_length: i+window_length+1, 0:number_of_features]

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(240, input_shape=(window_length, number_of_features), return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(240, input_shape=(window_length, number_of_features), return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(240, input_shape=(window_length, number_of_features), return_sequences=True)))
model.add(Bidirectional(LSTM(240, input_shape=(window_length, number_of_features), return_sequences=False)))
model.add(Dense(59))
model.add(Dense(number_of_features))

In [ ]:
model.compile(loss='mse', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train, label, epochs=100, batch_size=32, verbose=1)

# Prepare data for prediction
to_predict_1 = df.tail(100)
to_predict_1.drop([to_predict_1.index[-1]], axis=0, inplace=True)
to_predict_1 = np.array(to_predict_1)
scales_to_predict = scaler.transform(to_predict_1)


In [ ]:
# Obtain the precision of the model
accuracy = model.evaluate(train, label)
print('Accuracy: %.2f' % (accuracy[1]*100))

scaled_predicted_output = model.predict(np.array([scales_to_predict]))
predicted_output = scaler.inverse_transform(scaled_predicted_output).astype(int)[0]
print(predicted_output)